# Ejercicio 1 - Suma filas y columnas de matriz

Partiendo del código suministrado:

* Completar la función que suma las columnas de la matriz

* Añadir la invocación del nuevo `kernel col_sum` y la comprobación de resultados

* Comparar las prestaciones de la suma por filas y por columnas

---

In [1]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [2]:
import numpy as np
from numba import cuda

# ----------------------------------------------------------- #

# KERNEL SUMA FILAS
@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[idx][i]
    sums[idx] = sum

# KERNEL SUMA COLUMNAS
@cuda.jit
def col_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[i][idx]
    sums[idx] = sum

# ----------------------------------------------------------- #

# PARÁMETROS
n = 32768 # tamaño del lado de la matriz
threads_per_block = 256 # hilos
blocks = int(n / threads_per_block) # bloques

# ----------------------------------------------------------- #

# VARIABLES CPU
h_a = np.ones(n*n).reshape(n, n).astype(np.float32)

# VARIABLES GPU
d_a = cuda.to_device(h_a)
d_sum_rows = cuda.device_array(shape=(n,), dtype=np.float32) # para suma filas
d_sum_cols = cuda.device_array(shape=(n,), dtype=np.float32) # para suma cols

# ----------------------------------------------------------- #

# LANZA KERNEL - suma filas
row_sums[blocks, threads_per_block](d_a, d_sum_rows, n) # calcula suma de filas
h_sum_rows = d_sum_rows.copy_to_host() # pasa resultado a GPU

# COMPROBACIÓN - suma filas
suma_rows_truth = h_a.sum(axis=1) # suma filas real
np.testing.assert_equal(h_sum_rows,suma_rows_truth)

# MEDICIÓN - suma fias
# cpu
print("\nTiempo en CPU - suma filas:")
%timeit h_a.sum(axis=1)

# gpu
print("\nTiempo en GPU - suma filas:")
%timeit row_sums[blocks, threads_per_block](d_a, d_sum_rows, n); cuda.synchronize()

# ----------------------------------------------------------- #

# LANZA KERNEL - suma cols
col_sums[blocks, threads_per_block](d_a, d_sum_cols, n) # calcula suma de columnas
h_sum_cols = d_sum_cols.copy_to_host() # pasa resultado a GPU

# COMPROBACIÓN - suma cols
suma_cols_truth = h_a.sum(axis=0) # suma columnas real
np.testing.assert_equal(h_sum_rows,suma_cols_truth)

# MEDICIÓN - suma cols
# cpu
print("\nTiempo en CPU - suma columnas:")
%timeit h_a.sum(axis=0)

# gpu
print("\nTiempo en GPU - suma columnas:")
%timeit col_sums[blocks, threads_per_block](d_a, d_sum_cols, n); cuda.synchronize()



Tiempo en CPU - suma filas:
487 ms ± 9.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Tiempo en GPU - suma filas:
63.7 ms ± 55.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Tiempo en CPU - suma columnas:
665 ms ± 9.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Tiempo en GPU - suma columnas:
28.4 ms ± 59.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
